<a href="https://colab.research.google.com/github/erlonL/testes-inferencia/blob/main/Zscore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

X: média amostral

n: número de valores da amostra

alpha: nivel de significância

sigma: desvio padrão

mu: média populacional

H0: a amostra provém de uma população com determinada média (mu = mu0)

H1: contesta a hipótese nula (mu != mu0)

In [7]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import random

## Teste de Hipótese p/ Média Populacional Unilateral

In [3]:
X = 3.9
n = 42
alpha = 0.05
mu0 = 4.2
sigma = 1

In [5]:
Z = (X - mu0) / (sigma/(n)**(1/2))
print(Z)

-1.9442222095223598

In [2]:
ppf = stats.norm.ppf(alpha)

-1.6448536269514729

In [6]:
if Z < ppf:
    print("Com 95% de confiança, temos provas para rejeitar H0.")
else:
    print("Com 95% de confiança, não temos provas para rejeitar H0")

Temos provas para rejeitar H0


## Teste de Hipótese p/ Média Populacional Bilateral

In [ ]:
ppf = stats.norm.ppf(alpha/2)

In [ ]:
if Z < ppf:
    print("Com 95% de confiança, temos provas para rejeitar H0.")
else:
    print("Com 95% de confiança, não temos provas para rejeitar H0")

## Teste de Hipótese p/ Média (sigma desconhecido)

In [58]:
# exemplo

mu0 = 18
n = 25
X = 16.808
S = 2.733
alpha = '0.010'

In [56]:
Tcalc = (X - mu0) / (S / (n)**(1/2) )
print(Tcalc)

-2.1807537504573733


In [52]:
import pandas as pd
from scipy.stats import t as t_student

tabela_t_student = pd.DataFrame(
    [],
    index=[i for i in range(1, 31)],
    columns = [i / 100 for i in range(10, 0, -1)]
)

for index in tabela_t_student.index:
    for column in tabela_t_student.columns:
        tabela_t_student.loc[index, column] = t_student.ppf(1 - float(column) / 2, index)

index=[('Graus de Liberdade (n - 1)', i) for i in range(1, 31)]
tabela_t_student.index = pd.MultiIndex.from_tuples(index)

columns = [("{0:0.3f}".format(i / 100), "{0:0.3f}".format((i / 100) / 2)) for i in range(10, 0, -1)]
tabela_t_student.columns = pd.MultiIndex.from_tuples(columns)

tabela_t_student.rename_axis(['Unicaudal', 'Bicaudal'], axis=1, inplace = True)

In [53]:
tabela_t_student

Unicaudal                         0.100     0.090     0.080     0.070  \
Bicaudal                          0.050     0.045     0.040     0.035   
Graus de Liberdade (n - 1) 1   6.313752  7.026366  7.915815  9.057887   
                           2   2.919986  3.103977  3.319764  3.578247   
                           3   2.353363  2.470807  2.605427  2.762599   
                           4   2.131847    2.2261  2.332873  2.455892   
                           5   2.015048  2.097837  2.190958  2.297392   
                           6    1.94318  2.019201  2.104306  2.201059   
                           7   1.894579  1.966153  2.046011  2.136453   
                           8   1.859548  1.927986  2.004152  2.090166   
                           9   1.833113  1.899222  1.972653  2.055395   
                           10  1.812461  1.876774  1.948099  2.028327   
                           11  1.795885  1.858772  1.928427  2.006663   
                           12  1.782288  1.844015  1.912313  1.988934   
                           13  1.770933    1.8317  1.898874  1.974158   
                           14   1.76131  1.821267  1.887496  1.961656   
                           15   1.75305  1.812316  1.877739   1.95094   
                           16  1.745884  1.804553  1.869279  1.941654   
                           17  1.739607  1.797755  1.861875   1.93353   
                           18  1.734064  1.791754   1.85534  1.926362   
                           19  1.729133  1.786417   1.84953  1.919992   
                           20  1.724718   1.78164  1.844331  1.914292   
                           21  1.720743  1.777339  1.839651  1.909164   
                           22  1.717144  1.773447  1.835417  1.904524   
                           23  1.713872  1.769907  1.831567  1.900307   
                           24  1.710882  1.766675  1.828051  1.896457   
                           25  1.708141  1.763711  1.824828  1.892928   
                           26  1.705618  1.760983  1.821863  1.889682   
                           27  1.703288  1.758466  1.819126  1.886686   
                           28  1.701131  1.756134  1.816592  1.883912   
                           29  1.699127  1.753968  1.814238  1.881336   
                           30  1.697261  1.751952  1.812047  1.878938   

Unicaudal                          0.060      0.050      0.040      0.030  \
Bicaudal                           0.030      0.025      0.020      0.015   
Graus de Liberdade (n - 1) 1   10.578895  12.706205  15.894545  21.204949   
                           2    3.896425   4.302653   4.848732   5.642778   
                           3     2.95051   3.182446   3.481909   3.896046   
                           4    2.600762   2.776445   2.998528    3.29763   
                           5    2.421585   2.570582   2.756509   3.002875   
                           6    2.313263   2.446912   2.612242   2.828928   
                           7    2.240879   2.364624   2.516752   2.714573   
                           8    2.189155   2.306004   2.448985   2.633814   
                           9    2.150375   2.262157   2.398441   2.573804   
                           10   2.120234   2.228139   2.359315   2.527484   
                           11   2.096139   2.200985    2.32814   2.490664   
                           12   2.076441   2.178813   2.302722     2.4607   
                           13   2.060038   2.160369   2.281604   2.435845   
                           14   2.046169   2.144787   2.263781   2.414898   
                           15   2.034289    2.13145    2.24854   2.397005   
                           16      2.024   2.119905   2.235358   2.381545   
                           17   2.015002   2.109816   2.223845   2.368055   
                           18   2.007067   2.100922   2.213703    2.35618   
                           19   2.000017   2.093024   2.204701   2.345648   
                           20   1.993713   2.0

In [60]:
Tc = tabela_t_student[alpha].iloc[(n-1)-1][0]

In [61]:
if Tcalc < Tc:
    print("Não temos evidências para rejeitar H0")
else:
    print("temos evidências para rejeitar H0")

Não temos evidências para rejeitar H0
